# Define StateSchema

In [1]:
from langgraph.graph.message import add_messages
from langchain_core.messages import BaseMessage
from typing_extensions import Annotated, Sequence, TypedDict

class MessagesState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]

## Create a new Chat

In [2]:
from utils import ChatHistoryManager
chat = ChatHistoryManager.new_chat()
print(f"Chat ID:", chat.chat_id)

Chat ID: 79b63dc1-5e48-478c-8049-032a5c7da2dc


In [3]:
# ! pip install -U langchain-qdrant
from rag import TextDocumentRAG

In [7]:
from utils import (
    YTContentData,
    webvtt_2_str,
    download_youtube_audio_or_transcript,
)

yt_video_data: YTContentData = download_youtube_audio_or_transcript(
    video_url="https://www.youtube.com/watch?v=W3I3kAg2J7w&t=231s"
)

vtt_str = webvtt_2_str(vtt_file_path=yt_video_data.transcript_path)

print(vtt_str)

2025-07-07 10:38:23.459 | DEBUG    | utils.youtube:download_youtube_audio_or_transcript:133 - Cache exists, validating cache...
2025-07-07 10:38:23.460 | DEBUG    | utils.youtube:download_youtube_audio_or_transcript:145 - Cache contains the requested data. Using cache.


Last year, Grant gave the math department's Michael E. Moody
lecture on where math and physics collide.
He explored unexpected connections between kinematics, optics, and quantum computing.
Our students loved his talk, and the senior class voted
overwhelmingly to ask him back as commencement speaker.
Grant has said that he seeks to educate and inspire
the world with the beauty and the power of mathematics.
He certainly has inspired many of us here at Harvey Mudd.
Please join me in welcoming Grant Sanderson.
Thank you President Nempahrd for that very warm introduction and for inviting me.
And thank you to the class of 2024 for including me in such a special day.
I had the joy of getting to know many of you last year on this visit.
And I distinctly remember coming away with the feeling
that a future in your hands is a bright future indeed.
For those in the audience who don't know who I am,
I focus on making videos about mathematics with an emphasis on visualizations.
It's a weird job.
I 